#KMEANS


In [ ]:
import pandas as pd

df = pd.read_excel('/content/BASE_BASE (1).xlsx')

# Mostrar las primeras filas del DataFrame para revisar su estructura
df.head()

In [ ]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.cluster import KMeans
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
import numpy as np

# Selección de características relevantes
selected_features = [
    "SEXO", "EDAD", "PAIS_RESIDENCIA",
    "IMPORTANCIA_OFERTA_ALOJATIVA", "MEDIDA_LIMPIEZA",
    "NOCHES", "PERSONAS_TOTAL", "COSTE_ALOJ_EUROS"
]

# Filtrar datos y eliminar filas con valores faltantes
df_cleaned = df[selected_features + ["ALOJ_CATEG"]].dropna()

# Separar características y target (para análisis posterior)
X = df_cleaned[selected_features]
y = df_cleaned["ALOJ_CATEG"]

# Escalamiento para numéricas, codificación para categóricas
numeric_features = X.select_dtypes(include=["int64", "float64"]).columns
categorical_features = X.select_dtypes(include=["object"]).columns

preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), numeric_features),
        ("cat", OneHotEncoder(drop="first"), categorical_features),
    ]
)

# Definir el pipeline con K-Means
pipeline = Pipeline(
    steps=[("preprocessor", preprocessor), ("kmeans", KMeans(random_state=42))]
)

# Determinar el número óptimo de clusters usando el método del codo
inertia = []
K_range = range(2, 11)  # Probar entre 2 y 10 clusters

for k in K_range:
    pipeline.set_params(kmeans__n_clusters=k)
    pipeline.fit(X)
    inertia.append(pipeline.named_steps["kmeans"].inertia_)

# Graficar el método del codo
plt.figure(figsize=(8, 5))
plt.plot(K_range, inertia, marker="o")
plt.title("Método del Codo para determinar K óptimo")
plt.xlabel("Número de Clusters (K)")
plt.ylabel("Inercia")
plt.xticks(K_range)
plt.grid()
plt.show()


In [ ]:
# Aplicar K-Means con K=2 clusters
pipeline.set_params(kmeans__n_clusters=2)
pipeline.fit(X)

# Agregar los clusters al DataFrame original para análisis
df_cleaned["Cluster"] = pipeline.named_steps["kmeans"].labels_

# Visualizar la distribución de ALOJ_CATEG dentro de cada cluster
cluster_analysis = df_cleaned.groupby(["Cluster", "ALOJ_CATEG"]).size().unstack(fill_value=0)

# Mostrar resultados
cluster_analysis


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Preparar los datos
cluster_analysis.plot(kind="bar", stacked=True, figsize=(10, 6))
plt.title("Distribución de ALOJ_CATEG por Cluster")
plt.xlabel("Cluster")
plt.ylabel("Cantidad")
plt.xticks(rotation=0)
plt.legend(title="ALOJ_CATEG")
plt.show()


In [ ]:
sns.scatterplot(
    data=df_cleaned,
    x="EDAD",
    y="COSTE_ALOJ_EUROS",
    hue="Cluster",
    palette="viridis"
)
plt.title("Dispersión de EDAD vs COSTE_ALOJ_EUROS por Cluster")
plt.xlabel("Edad")
plt.ylabel("Costo del Alojamiento (€)")
plt.legend(title="Cluster")
plt.show()


In [ ]:






# Excluir variables no numéricas para calcular la matriz de correlación
numeric_features = df_cleaned.select_dtypes(include=["int64", "float64"])

# Calcular matriz de correlación y ordenar las correlaciones con ALOJ_CATEG
corr_matrix = numeric_features.corr()
correlations_with_target = corr_matrix["ALOJ_CATEG"].sort_values(ascending=False)


# Generar gráficos para analizar la distribución de variables clave con respecto a ALOJ_CATEG
import seaborn as sns
import matplotlib.pyplot as plt

variables_to_plot = ["COSTE_ALOJ_EUROS", "NOCHES", "EDAD", "PERSONAS_TOTAL", "IMPORTANCIA_OFERTA_ALOJATIVA"]

for var in variables_to_plot:
    plt.figure(figsize=(8, 5))
    sns.boxplot(x="ALOJ_CATEG", y=var, data=df_cleaned)
    plt.title(f"Distribución de {var} por ALOJ_CATEG")
    plt.xlabel("ALOJ_CATEG (0=Hotel, 1=Vivienda Turística)")
    plt.ylabel(var)
    plt.grid()
    plt.show()

In [ ]:

# Separar características (X) y objetivo (ALOJ_CATEG para análisis posterior)
X = df_cleaned[selected_features]

# Identificar columnas numéricas y categóricas
numeric_features = X.select_dtypes(include=["int64", "float64"]).columns
categorical_features = X.select_dtypes(include=["object"]).columns

# Definir preprocesador para escalamiento y codificación
preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), numeric_features),
        ("cat", OneHotEncoder(drop="first"), categorical_features),
    ]
)


In [ ]:
# Crear pipeline con preprocesamiento y K-Means
pipeline = Pipeline(
    steps=[("preprocessor", preprocessor), ("kmeans", KMeans(n_clusters=3, random_state=42))]
)

# Entrenar el modelo K-Means
pipeline.fit(X)

# Agregar etiquetas de cluster al DataFrame original
df_cleaned["Cluster"] = pipeline.named_steps["kmeans"].labels_


In [ ]:
# Analizar la distribución de ALOJ_CATEG dentro de cada cluster
cluster_analysis = df_cleaned.groupby(["Cluster", "ALOJ_CATEG"]).size().unstack(fill_value=0)
print(cluster_analysis)

# Graficar la distribución de ALOJ_CATEG por cluster
cluster_analysis.plot(kind="bar", stacked=True, figsize=(10, 6))
plt.title("Distribución de ALOJ_CATEG por Cluster")
plt.xlabel("Cluster")
plt.ylabel("Cantidad")
plt.xticks(rotation=0)
plt.legend(title="ALOJ_CATEG")
plt.show()


In [ ]:
# Graficar la dispersión de EDAD vs COSTE_ALOJ_EUROS coloreada por Cluster
sns.scatterplot(
    data=df_cleaned,
    x="EDAD",
    y="COSTE_ALOJ_EUROS",
    hue="Cluster",
    palette="viridis"
)
plt.title("Dispersión de EDAD vs COSTE_ALOJ_EUROS por Cluster")
plt.xlabel("Edad")
plt.ylabel("Costo del Alojamiento (€)")
plt.legend(title="Cluster")
plt.show()


In [ ]:
# Importar las librerías necesarias
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder

from sklearn.pipeline import Pipeline
from sklearn.cluster import KMeans
import plotly.express as px


# Leer el archivo cargado
df = pd.read_excel("/content/BASE_BASE (1).xlsx")  # Cambiar el nombre si es diferente

# Seleccionar características relevantes
selected_features = [
    "EDAD", "MEDIDA_LIMPIEZA",
    "NOCHES", "PERSONAS_TOTAL", "COSTE_ALOJ_EUROS", "ALOJ_CATEG"
]

# Limpiar y preparar los datos
df_cleaned = df[selected_features].dropna()

# Separar características y objetivo
X = df_cleaned.drop(columns=["ALOJ_CATEG"])
y = df_cleaned["ALOJ_CATEG"]

# Identificar columnas numéricas y categóricas
numeric_features = X.select_dtypes(include=["int64", "float64"]).columns
categorical_features = X.select_dtypes(include=["object"]).columns

# Aplicar normalización a las variables numéricas
scaler = MinMaxScaler()
X_normalized = X.copy()
X_normalized[numeric_features] = scaler.fit_transform(X[numeric_features])

# Crear pipeline con K-Means
pipeline = Pipeline(
    steps=[
        ("preprocessor", ColumnTransformer(
            transformers=[
                ("num", MinMaxScaler(), numeric_features),
                ("cat", OneHotEncoder(drop="first"), categorical_features),
            ]
        )),
        ("kmeans", KMeans(n_clusters=3, random_state=42))
    ]
)

# Entrenar el modelo
pipeline.fit(X)
X_normalized["Cluster"] = pipeline.named_steps["kmeans"].labels_

# Usar una muestra del 80% de X_normalized para graficar
X_sampled = X_normalized.sample(frac=0.1, random_state=42)

# Crear el gráfico 3D
fig = px.scatter_3d(
    X_sampled,
    x="EDAD",  # Variable para el eje X
    y="COSTE_ALOJ_EUROS",  # Variable para el eje Y
    z="PERSONAS_TOTAL",  # Variable para el eje Z
    color="Cluster",  # Cluster asignado por el modelo
    title="Visualización 3D de Clusters (Normalizado)",
    labels={"Cluster": "Cluster"},
)

# Mostrar el gráfico interactivo
fig.show()

In [ ]:
# Asignar el número de cluster asignado a cada registro
X_normalized['Cluster'] = pipeline.named_steps['kmeans'].labels_

# Mostrar el DataFrame con la nueva columna 'Cluster'
X_normalized.head()  # Esto mostrará las primeras filas del DataFrame con la columna 'Cluster'


In [ ]:
# Crear DataFrame para cada grupo de clusters (0, 1, 2)
df_cluster_0 = X_normalized[X_normalized['Cluster'] == 0]
df_cluster_1 = X_normalized[X_normalized['Cluster'] == 1]
df_cluster_2 = X_normalized[X_normalized['Cluster'] == 2]

print("DataFrame del Cluster 0:")
df_cluster_0.head()

In [ ]:
print("\nDataFrame del Cluster 1:")
df_cluster_1.head()

In [ ]:
print("\nDataFrame del Cluster 2:")
df_cluster_2.head()

In [ ]:
mean_cluster_0 = df_cluster_0.mean()
mean_cluster_1 = df_cluster_1.mean()
mean_cluster_2 = df_cluster_2.mean()

mean_cluster_0_df = mean_cluster_0.to_frame().T
mean_cluster_1_df = mean_cluster_1.to_frame().T
mean_cluster_2_df = mean_cluster_2.to_frame().T


print("Medias del Cluster 0:")
mean_cluster_0_df


In [ ]:
print("\nMedias del Cluster 1:")
mean_cluster_1_df

In [ ]:
print("\nMedias del Cluster 2:")
mean_cluster_2_df

In [ ]:
# Añadir la columna 'ALOJ_CATEG' de df_cleaned a X_normalized
X_normalized['ALOJ_CATEG'] = df_cleaned['ALOJ_CATEG']

# Contar la cantidad de datos clasificados por cada cluster (0, 1, 2) y su distribución por 'ALOJ_CATEG'
cluster_distribution = X_normalized.groupby(['Cluster', 'ALOJ_CATEG']).size().unstack(fill_value=0)

# Mostrar la distribución de registros en cada cluster según la variable objetivo
print("Distribución de datos en cada cluster según la variable objetivo 'ALOJ_CATEG':")
cluster_distribution

In [ ]:
# Importar las librerías necesarias
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

# Leer el archivo cargado
df = pd.read_excel("/content/BASE_BASE (1).xlsx")  # Cambiar el nombre si es diferente

# Seleccionar características relevantes
selected_features = [
    "EDAD", "MEDIDA_LIMPIEZA",
    "NOCHES", "PERSONAS_TOTAL", "COSTE_ALOJ_EUROS", "ALOJ_CATEG"
]

# Limpiar y preparar los datos
df_cleaned = df[selected_features].dropna()

# Separar características y objetivo
X = df_cleaned.drop(columns=["ALOJ_CATEG"])
y = df_cleaned["ALOJ_CATEG"]

# Identificar columnas numéricas
numeric_features = X.select_dtypes(include=["int64", "float64"]).columns

# Aplicar normalización a las variables numéricas
scaler = MinMaxScaler()
X_normalized = X.copy()
X_normalized[numeric_features] = scaler.fit_transform(X[numeric_features])

# Crear y entrenar el modelo K-Means
kmeans = KMeans(n_clusters=3, random_state=42)
X_normalized["Cluster"] = kmeans.fit_predict(X_normalized)

# Crear el diagrama de barras para los clusters
cluster_counts = X_normalized["Cluster"].value_counts()
plt.figure(figsize=(7, 5))
cluster_counts.plot(kind='bar', color='purple', edgecolor='black')
plt.title("Cantidad de elementos por Cluster")
plt.xlabel("Cluster")
plt.ylabel("Cantidad de elementos")
plt.xticks(rotation=0)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()


In [ ]:
# Importar las librerías necesarias
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

# Leer el archivo cargado
df = pd.read_excel("/content/BASE_BASE (1).xlsx")  # Cambiar el nombre si es diferente

# Seleccionar características relevantes
selected_features = [
    "EDAD", "MEDIDA_LIMPIEZA",
    "NOCHES", "PERSONAS_TOTAL", "COSTE_ALOJ_EUROS", "ALOJ_CATEG"
]

# Limpiar y preparar los datos
df_cleaned = df[selected_features].dropna()

# Separar características y objetivo
X = df_cleaned.drop(columns=["ALOJ_CATEG"])
y = df_cleaned["ALOJ_CATEG"]

# Identificar columnas numéricas
numeric_features = X.select_dtypes(include=["int64", "float64"]).columns

# Aplicar normalización a las variables numéricas
scaler = MinMaxScaler()
X_normalized = X.copy()
X_normalized[numeric_features] = scaler.fit_transform(X[numeric_features])

# Crear y entrenar el modelo K-Means
kmeans = KMeans(n_clusters=3, random_state=42)
X_normalized["Cluster"] = kmeans.fit_predict(X_normalized)

# Contar la cantidad de valores 0 y 1 de "ALOJ_CATEG" por cluster
cluster_category_counts = df_cleaned.groupby(X_normalized["Cluster"])["ALOJ_CATEG"].value_counts().unstack()

# Crear el gráfico de barras
cluster_category_counts.plot(
    kind="bar",
    stacked=True,
    figsize=(10, 6),
    color=["blue", "red"],  # Colores para las categorías 0 y 1
    edgecolor="black"
)
plt.title("Cantidad de 0 y 1 por Cluster")
plt.xlabel("Cluster")
plt.ylabel("Cantidad")
plt.xticks(rotation=0)
plt.legend(title="ALOJ_CATEG", labels=["0 (Hoteles)", "1 (Viviendas Turísticas)"])
plt.grid(axis="y", linestyle="--", alpha=0.7)
plt.show()


In [ ]:


# Método del codo para encontrar el número óptimo de clusters
inertia = []
for i in range(1, 11):
    kmeans = KMeans(n_clusters=i, random_state=42)
    kmeans.fit(X_normalized)
    inertia.append(kmeans.inertia_)

# Graficar el método del codo
plt.figure(figsize=(8, 6))
plt.plot(range(1, 11), inertia, marker='o')
plt.title('Método del Codo')
plt.xlabel('Número de Clusters')
plt.ylabel('Inercia')
plt.grid(True)
plt.show()

In [ ]:
# Importar las librerías necesarias
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.cluster import KMeans
import plotly.express as px


# Leer el archivo cargado
df = pd.read_excel("/content/BASE_BASE (1).xlsx")  # Cambiar el nombre si es diferente

# Seleccionar características relevantes
selected_features = [
    "EDAD", "MEDIDA_LIMPIEZA",
    "NOCHES", "PERSONAS_TOTAL", "COSTE_ALOJ_EUROS", "ALOJ_CATEG"
]

# Limpiar y preparar los datos
df_cleaned = df[selected_features].dropna()

# Separar características y objetivo
X = df_cleaned.drop(columns=["ALOJ_CATEG"])
y = df_cleaned["ALOJ_CATEG"]

# Identificar columnas numéricas y categóricas
numeric_features = X.select_dtypes(include=["int64", "float64"]).columns
categorical_features = X.select_dtypes(include=["object"]).columns

# Aplicar normalización a las variables numéricas
scaler = MinMaxScaler()
X_normalized = X.copy()
X_normalized[numeric_features] = scaler.fit_transform(X[numeric_features])

# Crear pipeline con K-Means
pipeline = Pipeline(
    steps=[
        ("preprocessor", ColumnTransformer(
            transformers=[
                ("num", MinMaxScaler(), numeric_features),
                ("cat", OneHotEncoder(drop="first"), categorical_features),
            ]
        )),
        ("kmeans", KMeans(n_clusters=3, random_state=42))
    ]
)

# Entrenar el modelo
pipeline.fit(X)
X_normalized["Cluster"] = pipeline.named_steps["kmeans"].labels_

# Usar una muestra del 80% de X_normalized para graficar
X_sampled = X_normalized.sample(frac=0.1, random_state=42)

# Crear el gráfico 3D
fig = px.scatter_3d(
    X_sampled,
    x="EDAD",  # Variable para el eje X
    y="COSTE_ALOJ_EUROS",  # Variable para el eje Y
    z="PERSONAS_TOTAL",  # Variable para el eje Z
    color="Cluster",  # Cluster asignado por el modelo
    title="Visualización 3D de Clusters (Normalizado)",
    labels={"Cluster": "Cluster"},
)

# Mostrar el gráfico interactivo
fig.show()

Dataframe por clúster


In [ ]:
# Crear un DataFrame para el cluster 1
df_cluster_1 = df_cleaned[df_cleaned["Cluster"] == 1]

# Crear un DataFrame para el cluster 0
df_cluster_0 = df_cleaned[df_cleaned["Cluster"] == 0]

print("DataFrame para Cluster 1:")
df_cluster_1

In [ ]:
print("DataFrame para Cluster 0:")
df_cluster_0

In [ ]:
df_cluster_1['SEXO'].value_counts()

In [ ]:
mean_cluster_0 = df_cluster_0.mean()
print("Mean of columns for df_cluster_0:")
mean_cluster_0


In [ ]:
df_cluster_0['SEXO'].value_counts()

In [ ]:
mean_cluster_1 = df_cluster_1.mean()
print("\nMean of columns for df_cluster_1:")
mean_cluster_1

In [ ]:
pca = PCA(n_components=2)
X_pca = pca.fit_transform(pipeline.named_steps["preprocessor"].transform(X))

df_cleaned["PCA1"] = X_pca[:, 0]
df_cleaned["PCA2"] = X_pca[:, 1]

plt.figure(figsize=(10, 6))
sns.scatterplot(
    x="PCA1", y="PCA2", hue="Cluster", data=df_cleaned, palette="viridis", s=60
    )
    plt.title("Visualización de Clusters (PCA)")
    plt.xlabel("Componente Principal 1")
    plt.ylabel("Componente Principal 2")
    plt.legend(title="Cluster")
    plt.grid()
    plt.show()


In [ ]:



# Preparar los datos
cluster_analysis.plot(kind="bar", stacked=True, figsize=(10, 6))
plt.title("Distribución de ALOJ_CATEG por Cluster")
plt.xlabel("Cluster")
plt.ylabel("Cantidad")
plt.xticks(rotation=0)
plt.legend(title="ALOJ_CATEG")
plt.show()


In [ ]:
sns.scatterplot(
      data=df_cleaned,
          x="EDAD",
              y="COSTE_ALOJ_EUROS",
                  hue="Cluster",
                      palette="viridis"
                      )
                      plt.title("Dispersión de EDAD vs COSTE_ALOJ_EUROS por Cluster")
                      plt.xlabel("Edad")
                      plt.ylabel("Costo del Alojamiento (€)")
                      plt.legend(title="Cluster")
                      plt.show()
)

In [ ]:
# Graficar el método del codo
plt.figure(figsize=(8, 5))
plt.plot(K_range, inertia, marker="o")
plt.title("Método del Codo para determinar K óptimo")
plt.xlabel("Número de Clusters (K)")
plt.ylabel("Inercia")
plt.xticks(K_range)
plt.grid()
plt.show()